# GBM(Gradient Boosting Machine)

## get_human_dataset()

In [20]:
import pandas as pd
import matplotlib.pyplot as plt


# Load data
file_dir = 'UCI HAR Dataset/UCI HAR Dataset'
feature_name_df = pd.read_csv(file_dir + '/features.txt', sep='\s+',
                             header=None, names=['column_index', 'column_name'])

# 피처명 index 제거, 피처명만 리스트 객체로 생성한 뒤 샘플로 10개만 추출
feature_name = feature_name_df.iloc[:, 1].values.tolist()
print('전체 피처명에서 10개만 추출:', feature_name[:10])

전체 피처명에서 10개만 추출: ['tBodyAcc-mean()-X', 'tBodyAcc-mean()-Y', 'tBodyAcc-mean()-Z', 'tBodyAcc-std()-X', 'tBodyAcc-std()-Y', 'tBodyAcc-std()-Z', 'tBodyAcc-mad()-X', 'tBodyAcc-mad()-Y', 'tBodyAcc-mad()-Z', 'tBodyAcc-max()-X']


In [21]:
# 함수정의
def get_new_feature_name_df(old_feature_name_df):
    feature_dup_df = pd.DataFrame(data=old_feature_name_df.groupby('column_name').cumcount(), columns=['dup_cnt'])
    feature_dup_df = feature_dup_df.reset_index()
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(), feature_dup_df, how='outer')
    new_feature_name_df['column_name'] = new_feature_name_df[['column_name', 'dup_cnt']].apply(lambda x:x[0] + '_' + str(x[1])
                                                                                              if x[1] > 0 else x[0], axis=1)
    new_feature_name_df = new_feature_name_df.drop(['index'], axis=1)
    return new_feature_name_df

def get_human_dataset():
    global file_dir
    # 각 데이터 파일은 공백으로 분리되어 있으므로 read_csv에서 공백 문자를 sep으로 할당.
    feature_name_df = pd.read_csv(file_dir + '/features.txt', sep='\s+',
                                 header=None, names=['column_index', 'column_name'])
    
    # 중복된 피처명을 수정하는 get_new_feature_name_df() 를 이용, 신규 피처명 DataFrame 생성.
    new_feature_name_df = get_new_feature_name_df(feature_name_df)
    
    # DataFrame에 피처명을 칼럼으로 부여하기 위해 리스트 객체로 다시 변환
    feature_name = new_feature_name_df.iloc[:, 1].values.tolist()
    
    # 학습 피처 데이터셋과 테스트 피처 데이터를 DataFrame으로 로딩. 칼럼명은 feature_name 적용
    X_train = pd.read_csv(file_dir + '/train/X_train.txt', sep='\s+', header=None, names=feature_name)
    X_test = pd.read_csv(file_dir + '/test/X_test.txt', sep='\s+', header=None, names=feature_name)
    
    Y_train = pd.read_csv(file_dir + '/train/Y_train.txt', sep='\s+', header=None, names=['action'])
    Y_test = pd.read_csv(file_dir + '/test/Y_test.txt', sep='\s+', header=None, names=['action'])
    
    return X_train, X_test, Y_train, Y_test

In [22]:
from sklearn.ensemble import GradientBoostingClassifier
import time
import warnings
warnings.filterwarnings('ignore')

X_train, X_test, Y_train, Y_test = get_human_dataset()
print(X_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7352 entries, 0 to 7351
Columns: 561 entries, tBodyAcc-mean()-X to angle(Z,gravityMean)
dtypes: float64(561)
memory usage: 31.5 MB
None


In [23]:
# GBM 수행 시간 측정을 위함. 시작 시간 설정
start_time = time.time()

In [24]:
from sklearn.metrics import accuracy_score
gb_clf = GradientBoostingClassifier(random_state=0)
gb_clf.fit(X_train, Y_train)
gb_pred = gb_clf.predict(X_test)
gb_accuracy = accuracy_score(Y_test, gb_pred)

print(f'GBM 정확도: {gb_accuracy:4f}')
print('GBM 수행 시간: {0:.1f} 초'.format(time.time() - start_time))

GBM 정확도: 0.938242
GBM 수행 시간: 762.0 초


In [25]:
GradientBoostingClassifier??

In [26]:
print(GradientBoostingClassifier.__doc__)

Gradient Boosting for classification.

    GB builds an additive model in a
    forward stage-wise fashion; it allows for the optimization of
    arbitrary differentiable loss functions. In each stage ``n_classes_``
    regression trees are fit on the negative gradient of the
    binomial or multinomial deviance loss function. Binary classification
    is a special case where only a single regression tree is induced.

    Read more in the :ref:`User Guide <gradient_boosting>`.

    Parameters
    ----------
    loss : {'deviance', 'exponential'}, optional (default='deviance')
        loss function to be optimized. 'deviance' refers to
        deviance (= logistic regression) for classification
        with probabilistic outputs. For loss 'exponential' gradient
        boosting recovers the AdaBoost algorithm.

    learning_rate : float, optional (default=0.1)
        learning rate shrinks the contribution of each tree by `learning_rate`.
        There is a trade-off between learning_ra

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators': [100, 500],
    'learning_rate': [0.05, 0.01]
}
grid_cv = GridSearchCV(gb_clf, param_grid=params, cv=2, verbose=1)
grid_cv.fit(X_train, Y_train)
print('최적 하이퍼 파라미터: \n', grid_cv.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))

Fitting 2 folds for each of 4 candidates, totalling 8 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


In [ ]:
# GridSearchCv 를 이용해 최적으로 학습된 estimator로 예측 수행.
gb_pred = grid_cv.best_estimator_.predict(X_test)
gb_accuracy = accuracy_score(Y_test, gb_pred)
print('GBM 정확도: {0:4f}'.format(gb_accuracy))

---

## XGBoost(eXtra Gradient Boost)

### XGBOOST 개요
XGBoost는 트리 기반의 앙상블 학습에서 가장 각광받고 있는 알고리즘 중 하나이다. 유명한 캐글 경연 대회`(Kaggle Contest)`에서 상위를 차지한 많은 데이터 과학자가 XGBoost를 이용하면서 널리 알려졌다.  
압도적인 수치의 차이는 아니지만, 분류에 있어서 일반적으로 다른 머신러닝보다 뛰어난 예측 성능을 나타낸다.  

- XGBoost는 GBM에 기반하고 있지만, GBM의 단점인 느린 수행 시간 및 과적합 규제(Regularization) 부재 등의 문제를 해결해서 각광을 받고 있다.
- 특히 별렬 CPU 환경에서 병렬 학습이 가능해 기존 GBM보다 빠르게 학습을 완료할 수 있는 것이 주요 장점이다.

> - 일반적인 GBM은 순차적으로 Weak learner가 가중치를 증감하는 방법으로 학습하기때문에 전반적으로 속도가 느리다. 하지만 XGBoost는 병렬 수행 및 다양한 기능으로 GBM에 비해 빠른 수행 성능을 보장한다.
> - 단, XGBoost가 GBM에 비해 수행 시간이 빠르다는 것이지, 다른 머신러닝 알고리즘(예를 들어 Random Forest)에 비해 바르다는 의미는 아니다.


In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier

In [ ]:
print(XGBClassifier.__doc__)

In [ ]:
print(xgboost.__version__)

In [ ]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

dataset = load_breast_cancer()
X_features = dataset.data
y_label = dataset.target
cancer_df = pd.DataFrame(data=X_features, column=dataset.feature_names)
cancer_df['target'] = y_label
cancer_df.head(3)

In [ ]:
print(dataset.target_names)
print(cancer_df['target'].value_counts())

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X_features, y_label, test_size=0.2, random_state=156)
print(X_train.shape, X_test.shape)

In [ ]:
dtrain = xgb.DMatrix(data=X_train, label=Y_train)
dtest = xgb.DMatrix(data=X_test, label=Y_test)

In [ ]:
wlist = [(dtrain, 'train'), (dtest, 'eval')]
# 하이퍼 파라미터와 early stopping 파라미터를 train()함수의 파라미터로 전달
xgb_model =xgb.train(params = params, dtrain=dtrain, num_boost_round = num_rounds,
                     early_stopping_rounds = 100, evals=wlist)

In [ ]:
from xgboost import plot_importance